In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize, basinhopping
%matplotlib inline

In [22]:
df = pd.read_excel('../data/asset.xlsx', index_col=0)

In [23]:
df.tail()

,000016.SH,000906.SH,HSI.HI,IXIC.GI
2017-08-22,2643.5032,4205.2359,27401.67,6297.48
2017-08-23,2659.7804,4198.4550,27401.67,6278.41
2017-08-24,2646.4016,4172.9899,27518.60,6271.33
2017-08-25,2706.2421,4234.8388,27848.16,6265.64
2017-08-28,2736.4617,4287.1433,27863.29,6265.64


In [24]:
ret_df = df.pct_change()

In [25]:
cov = ret_df.cov()

In [26]:
# 风险预算优化
def calculate_portfolio_var(w, V):
    # 计算组合风险的函数
    return w.dot(V).dot(w)

def calculate_risk_contribution(w ,V):
    # 边际风险贡献
    MRC = V.dot(w)
    # 风险贡献
    RC = w * MRC
    return RC

def risk_budget_objective(x, pars):
    # 计算组合风险
    V = pars[0]# 协方差矩阵
    MRC = V.dot(x)
    ARC = x * MRC
    J = np.sum([(i-j)**2 for i in ARC for j in ARC])
#     J = x.dot(MRC) - np.log(x).sum()
#     J = calculate_portfolio_var(x, V) - np.log(x).sum()
#     J = sum(np.square(asset_RC - risk_target.T))[0, 0] # sum of squared error
#     J = np.abs(asset_RC.max() - asset_RC.min())
    return J

def total_weight_constraint(x):
    return np.sum(x) - 1.0
    
def long_only_constraint(x):
    return x
    
# 根据资产预期目标风险贡献度来计算各资产的权重
def calcu_w(V):
    N = V.shape[0]
    w0 = np.random.random_sample((N,))
    w0 /= w0.sum()
    cons = ({'type': 'eq', 'fun': total_weight_constraint},
            {'type': 'ineq', 'fun': long_only_constraint})
    minimizer_args = {'method': 'SLSQP', 'args': [V], 'constraints': cons}
#     minimizer_args = {'method': 'L-BFGS-B', 'args': [V], 'bounds': np.array([(0., None)]*N)}
#     weight = res.x / res.x.sum()
    res = basinhopping(risk_budget_objective, w0, stepsize=1e-8, minimizer_kwargs=minimizer_args)
#     res = minimize(risk_budget_objective, w0, args=[V], method='SLSQP',constraints=cons)
    return res.x

In [27]:
weight = calcu_w(cov*10000)

In [28]:
weight

array([ 0.18185088,  0.18400344,  0.2687741 ,  0.36537158])

In [29]:
calculate_risk_contribution(weight, cov)

000016.SH    0.000021
000906.SH    0.000021
HSI.HI       0.000021
IXIC.GI      0.000021
dtype: float64

In [33]:
ret_df.iloc[-242:].corr()

,000016.SH,000906.SH,HSI.HI,IXIC.GI
000016.SH,1.000000,0.715839,0.447734,0.106468
000906.SH,0.715839,1.000000,0.467823,0.060985
HSI.HI,0.447734,0.467823,1.000000,0.111086
IXIC.GI,0.106468,0.060985,0.111086,1.000000
